In [1]:
import numpy as np
import pandas as pd

from os import listdir
from numpy import asarray
from numpy import save
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
# from tqdm import tqdm

from os import makedirs
from os import listdir
from random import random
from shutil import copyfile
import random

import sys
# from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# src = 'train/'
# dataset = 'CatDogDF/'
# subdirs = ['train/', 'val/']
# labeldirs = ['cats/', 'dogs/']

# for subdir in subdirs:
#     for labeldir in labeldirs:
#         newdir = dataset + subdir + labeldir
#         makedirs(newdir, exist_ok = True)

In [6]:
# random.seed(5)
# val_ratio = 0.2
# files = listdir(src)
# # print(len(files))
# random.shuffle(files)

# for file in tqdm(files):
#     srcpath = src + file
#     if random.random() < val_ratio: subfolder1 = 'val/'
#     else: subfolder1 = 'train/'
#     if file.startswith('cat'): subfolder2 = 'cats/'
#     else: subfolder2 = 'dogs/'
#     destpath = 'CatDogDF/' + subfolder1 + subfolder2 + file
#     copyfile(srcpath, destpath)

100%|██████████████████████████████████████████████████████████████████████████| 25000/25000 [00:19<00:00, 1273.28it/s]


In [6]:
# def summarize_diagnostics(history):
#     pyplot.subplot(211)
#     pyplot.title('Cross Entropy Loss')
#     pyplot.plot(history.history['loss'], color='blue', label='train')
#     pyplot.plot(history.history['val_loss'], color='orange', label='test')
#     # plot accuracy
#     pyplot.subplot(212)
#     pyplot.title('Classification Accuracy')
#     pyplot.plot(history.history['accuracy'], color='blue', label='train')
#     pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
#     pyplot.show()

In [2]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding= 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding= 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding= 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding= 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
# model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding= 'same'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(300, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 128)       7

In [5]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range= 20, width_shift_range=0.1, height_shift_range=0.1, 
                             zoom_range=0.1, horizontal_flip= True)
# datagen = ImageDataGenerator(rescale=1.0/255.0)
# prepare iterators
train_it = datagen.flow_from_directory('CatDogDF/train/', class_mode='binary', batch_size=128,
                                       target_size=(200, 200), shuffle = True)
test_it = datagen.flow_from_directory('CatDogDF/val/', class_mode='binary', batch_size=128,
                                      target_size=(200, 200), shuffle = True)

Found 20014 images belonging to 2 classes.
Found 4986 images belonging to 2 classes.


In [ ]:
history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=test_it,
                              validation_steps=len(test_it), epochs=100, verbose=1)


# evaluate model
# _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=2)
# print('> %.3f' % (acc * 100.0))

# learning curves
# summarize_diagnostics(history)

Epoch 1/100
157/157 [==============================] - 319s 2s/step - loss: 0.6902 - accuracy: 0.5301 - val_loss: 0.6914 - val_accuracy: 0.5471
Epoch 2/100
 47/157 [=======>......................] - ETA: 3:05 - loss: 0.6868 - accuracy: 0.5444

In [14]:
from tensorflow.python.client import device_lib
import tensorflow

def device_status():

    available_devices = device_lib.list_local_devices()

    print('\n\n\n System Information')
    print('===============================================================')
    print(f'Tesorflow Version: {tensorflow.__version__}')
    gpu_flag = 0
    for device in available_devices:

        if (device.device_type == 'GPU'):

            print(f'Availabe GPU Status: {device}')
            gpu_flag = gpu_flag + 1

    if (gpu_flag==0):
        print('No GPU detected. Running on CPU')

    print('===============================================================')
    print('\n')

    
device_status()




 System Information
Tesorflow Version: 2.5.0
Availabe GPU Status: name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3660972032
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16425326855962268098
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"





In [16]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4250960102226021309,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3660972032
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 25280257675146684
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"]